<a href="https://colab.research.google.com/github/vsukamet/NLPProject/blob/master/USE_Word_EMbeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 512kB 44.6MB/s 
     |████████████████████████████████| 3.8MB 43.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=89170c527cb38fe681564b1678f8f4426aa71c9c122d1a583dccb228a883a60e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installatio

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


In [0]:
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

In [0]:
import json
import requests
df = json.loads(requests.get('https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_oos_plus.json').text)

In [0]:

''' Getting Training data '''
inscope_train = pd.DataFrame(df['train'],columns=['query','intent'])
oss_train = pd.DataFrame(df['oos_train'],columns=['query','intent'])

''' Getting Validation data '''
inscope_val = pd.DataFrame(df['val'],columns=['query','intent'])
oss_val = pd.DataFrame(df['oos_val'],columns=['query','intent'])

''' Getting Test data '''
inscope_test = pd.DataFrame(df['test'],columns=['query','intent'])
oss_test = pd.DataFrame(df['oos_test'],columns=['query','intent'])


train_df = pd.concat([inscope_train,oss_train])
val_df = pd.concat([inscope_val,oss_val])
test_df = pd.concat([inscope_test,oss_test])

In [0]:
train_df = train_df.append(val_df).reset_index(drop=True)
x = train_df.append(test_df).reset_index(drop=True)

X = x['query'].values
Y = x['intent'].values

In [49]:
le = preprocessing.LabelEncoder()
le.fit(Y)
le.classes_

array(['accept_reservations', 'account_blocked', 'alarm',
       'application_status', 'apr', 'are_you_a_bot', 'balance',
       'bill_balance', 'bill_due', 'book_flight', 'book_hotel',
       'calculator', 'calendar', 'calendar_update', 'calories', 'cancel',
       'cancel_reservation', 'car_rental', 'card_declined', 'carry_on',
       'change_accent', 'change_ai_name', 'change_language',
       'change_speed', 'change_user_name', 'change_volume',
       'confirm_reservation', 'cook_time', 'credit_limit',
       'credit_limit_change', 'credit_score', 'current_location',
       'damaged_card', 'date', 'definition', 'direct_deposit',
       'directions', 'distance', 'do_you_have_pets', 'exchange_rate',
       'expiration_date', 'find_phone', 'flight_status', 'flip_coin',
       'food_last', 'freeze_account', 'fun_fact', 'gas', 'gas_type',
       'goodbye', 'greeting', 'how_busy', 'how_old_are_you',
       'improve_credit_score', 'income', 'ingredient_substitution',
       'ingredients_l

In [0]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [0]:
test = encode(le, ['order_checks', 'travel_suggestion', 'uber', 'what_is_your_name'])


In [52]:
untest = decode(le, test)
untest

array(['order_checks', 'travel_suggestion', 'uber', 'what_is_your_name'],
      dtype=object)

In [53]:
x_enc = X
y_enc = encode(le, Y)
x_enc.shape

(23850,)

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_enc, y_enc, test_size = 0.3, random_state = 0)

In [0]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [0]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

In [34]:
MAX_SEQUENCE_LENGTH = 150
input_text = Input(shape=(1,),dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(151, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [35]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=7, batch_size=32)
    model.save_weights('./model.h5')

Epoch 1/7
16695/16695 [==============================] - 180s 11ms/step - loss: 2.0839 - accuracy: 0.5963
Epoch 2/7
16695/16695 [==============================] - 179s 11ms/step - loss: 0.4592 - accuracy: 0.9010
Epoch 3/7
16695/16695 [==============================] - 180s 11ms/step - loss: 0.3025 - accuracy: 0.9263
Epoch 4/7
16695/16695 [==============================] - 180s 11ms/step - loss: 0.2335 - accuracy: 0.9418
Epoch 5/7
16695/16695 [==============================] - 180s 11ms/step - loss: 0.1938 - accuracy: 0.9518
Epoch 6/7
16695/16695 [==============================] - 179s 11ms/step - loss: 0.1628 - accuracy: 0.9593
Epoch 7/7
16695/16695 [==============================] - 180s 11ms/step - loss: 0.1410 - accuracy: 0.9652


In [0]:
 with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)

In [0]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [0]:
from sklearn import metrics

boolean_test = (y_test != 'oos')
y_test_inclass = y_test[y_test != 'oos']
y_test_pred_inclass = [y_preds[i] for i in range(len(y_preds)) if boolean_test[i]]

print("Inscope_Accuracy using SVM (USE Embeddings)",metrics.accuracy_score(y_true = y_test_inclass,y_pred = y_test_pred_inclass))

In [0]:
from collections import Counter

true_positives_test = Counter(np.logical_and(y_preds == 'oos',y_test == 'oos'))[True]
false_negatives_test = Counter(np.logical_and(y_preds != 'oos',y_test == 'oos'))[True]

test_recall = (true_positives_test)/(true_positives_test+false_negatives_test)

print("Out_of_Scope_Recall",test_recall)

In [0]:
from sklearn import metrics

# _,INSCOPE_ACCURACY = model.evaluate()
print(metrics.confusion_matrix(y_test, y_preds))

print('Inscope-accuracy:',metrics.accuracy_score(y_test, y_preds))
print(metrics.classification_report(y_test, y_preds))